# 13 - Create Feature Descriptions

In [1]:
from collections import Counter
import json
import os

In [2]:
role_skills = {
    'Production Technician I': [
        ('Basic Machinery Maintenance', lambda exp: 0.9 if exp <= 5 else 0.6),
        ('Safety Protocols', lambda exp: 0.9 if exp <= 5 else 0.7),
        ('Problem Identification', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('Advanced Machinery Troubleshooting', lambda exp: 0.2 if exp <= 5 else 0.6),
        ('Teamwork', lambda exp: 0.8),
    ],
    'Production Technician II': [
        ('Advanced Machinery Maintenance', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Safety Protocols', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Problem-Solving', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Efficiency Optimization', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Leadership Skills', lambda exp: 0.2 if exp <= 5 else 0.6),
        ('Teamwork', lambda exp: 0.8),
        ('Advanced Troubleshooting Techniques', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Preventive Maintenance Planning', lambda exp: 0.4 if exp <= 5 else 0.7),
    ],
    'Area Sales Manager': [
        ('Sales Strategy', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Negotiation', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Customer Relationship Management', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Market Analysis', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Team Leadership', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Advanced CRM Tools', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Competitor Analysis', lambda exp: 0.4 if exp <= 5 else 0.6),
    ],
    'Production Manager': [
        ('Process Optimization', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Team Management', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Budget Planning', lambda exp: 0.4 if exp <= 5 else 0.8),
        ('Lean Manufacturing', lambda exp: 0.3 if exp <= 5 else 0.7),
        ('Quality Assurance', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Production Line Efficiency Analysis', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Cost Reduction Techniques', lambda exp: 0.4 if exp <= 5 else 0.7),
    ],
    'IT Support': [
        ('Troubleshooting', lambda exp: 0.9),
        ('Hardware Maintenance', lambda exp: 0.8),
        ('Customer Support', lambda exp: 0.7),
        ('Network Configuration', lambda exp: 0.3 if exp <= 5 else 0.7),
        ('System Upgrades', lambda exp: 0.2 if exp <= 5 else 0.6),
    ],
    'Software Engineer': [
        ('Python', lambda exp: 0.9 if exp <= 5 else 0.95),
        ('Java', lambda exp: 0.7 if exp <= 5 else 0.85),
        ('Software Design', lambda exp: 0.6 if exp <= 5 else 0.9),
        ('System Architecture', lambda exp: 0.2 if exp <= 5 else 0.8),
        ('Machine Learning', lambda exp: 0.3 if exp <= 5 else 0.7),
    ],
    'Data Analyst': [
        ('SQL', lambda exp: 0.9 if exp <= 5 else 0.95),
        ('Python', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Data Visualization', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('Statistical Analysis', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Business Intelligence Tools', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Machine Learning', lambda exp: 0.3 if exp <= 5 else 0.7),
    ],
    'Network Engineer': [
        ('Network Configuration', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Firewall Management', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('VPN Setup', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Troubleshooting', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Cloud Networking', lambda exp: 0.3 if exp <= 5 else 0.7),
        ('Advanced Firewall Configurations', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Network Performance Optimization', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('SD-WAN Deployment', lambda exp: 0.3 if exp <= 5 else 0.6),
    ],
    'Database Administrator': [
        ('SQL Optimization', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Database Management', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Backup Strategies', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Performance Tuning', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Data Security', lambda exp: 0.3 if exp <= 5 else 0.6),
    ],
    'Sales Manager': [
        ('Sales Strategy', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Team Management', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('Negotiation', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Customer Retention', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Revenue Optimization', lambda exp: 0.3 if exp <= 5 else 0.7),
    ],
    'Sr. Network Engineer': [
        ('Advanced Network Configuration', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Firewall Expertise', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Cloud Integration', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Network Security Design', lambda exp: 0.3 if exp <= 5 else 0.7),
        ('System Troubleshooting', lambda exp: 0.7),
    ],
    'BI Developer': [
        ('Data Visualization', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('ETL Development', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('Business Intelligence Tools', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('SQL', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Dashboard Creation', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('ETL Automation', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Predictive Analytics Integration', lambda exp: 0.4 if exp <= 5 else 0.6),
    ],
    'Administrative Assistant': [
        ('Document Management', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Scheduling', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('Customer Communication', lambda exp: 0.6 if exp <= 5 else 0.7),
        ('Basic Accounting', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('Office Coordination', lambda exp: 0.5 if exp <= 5 else 0.7),
    ],
    'Accountant I': [
        ('Financial Reporting', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('QuickBooks', lambda exp: 0.6 if exp <= 5 else 0.7),
        ('Tax Preparation', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Budget Planning', lambda exp: 0.3 if exp <= 5 else 0.6),
        ('Audit Assistance', lambda exp: 0.5),
    ],
    'Enterprise Architect': [
        ('System Architecture Design', lambda exp: 0.8 if exp <= 5 else 0.95),
        ('Solution Architecture', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Business-IT Alignment', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Cloud Strategy', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Governance and Standards', lambda exp: 0.3 if exp <= 5 else 0.6),
    ],
    'IT Director': [
        ('IT Governance', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Leadership', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Vendor Management', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('IT Security Oversight', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Strategic Planning', lambda exp: 0.4 if exp <= 5 else 0.7),
    ],
    'President & CEO': [
        ('Strategic Vision', lambda exp: 0.9),
        ('Leadership', lambda exp: 0.8),
        ('Financial Management', lambda exp: 0.7),
        ('Public Relations', lambda exp: 0.6),
        ('Risk Assessment', lambda exp: 0.5),
    ],
    'IT Manager - DB': [
        ('Database Management', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Performance Tuning', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Backup Strategies', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Vendor Management', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('IT Governance', lambda exp: 0.3 if exp <= 5 else 0.6),
    ],
    'Administrative Assistant': [
        ('Document Management', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Scheduling', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('Customer Communication', lambda exp: 0.6 if exp <= 5 else 0.7),
        ('Office Coordination', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Basic Accounting', lambda exp: 0.4 if exp <= 5 else 0.6),
    ],
    'Senior BI Developer': [
        ('Data Pipeline Optimization', lambda exp: 0.8 if exp <= 5 else 0.95),
        ('Advanced Data Visualization', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('ETL Development', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('SQL', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Machine Learning Integration', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Real-Time Data Processing', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Advanced Predictive Modeling', lambda exp: 0.3 if exp <= 5 else 0.6),
        ('ETL Automation', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Predictive Analytics Integration', lambda exp: 0.4 if exp <= 5 else 0.7),
    ],
    'Shared Services Manager': [
        ('Process Improvement', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Team Coordination', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Service Delivery Optimization', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Vendor Management', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('Leadership', lambda exp: 0.3 if exp <= 5 else 0.7),
    ],
    'CIO': [
        ('IT Governance', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Strategic Vision', lambda exp: 0.6 if exp <= 5 else 0.9),
        ('Cybersecurity Oversight', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Vendor Management', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Leadership', lambda exp: 0.9),
        ('Technology Roadmap Development', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Strategic IT Investment Planning', lambda exp: 0.3 if exp <= 5 else 0.6),
    ],
    'IT Manager - Support': [
        ('IT Support Management', lambda exp: 0.8 if exp <= 5 else 0.9),
        ('Troubleshooting Oversight', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('Customer Support Strategies', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Hardware Management', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Team Leadership', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Advanced ITSM Tools', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Incident Response Planning', lambda exp: 0.3 if exp <= 5 else 0.6),
    ],
    'IT Manager - Infra': [
        ('Infrastructure Design', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Network Management', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('System Upgrades', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Cloud Integration', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Leadership', lambda exp: 0.3 if exp <= 5 else 0.7),
        ('Hybrid Cloud Infrastructure Management', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Disaster Recovery Planning', lambda exp: 0.3 if exp <= 5 else 0.6),
    ],
    'Data Architect': [
        ('Data Modeling', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Database Design', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Big Data Solutions', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Cloud Data Management', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('ETL Optimization', lambda exp: 0.3 if exp <= 5 else 0.6),
        ('Data Lake Architecture', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Data Pipeline Scalability', lambda exp: 0.4 if exp <= 5 else 0.6),
    ],
    'BI Director': [
        ('Business Intelligence Strategy', lambda exp: 0.6 if exp <= 5 else 0.9),
        ('Data Governance', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Advanced Visualization', lambda exp: 0.4 if exp <= 5 else 0.8),
        ('Team Leadership', lambda exp: 0.8),
        ('Process Optimization', lambda exp: 0.3 if exp <= 5 else 0.7),
    ],
    'Director of Sales': [
        ('Sales Strategy', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Customer Relationship Management', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Revenue Optimization', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Market Analysis', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Team Leadership', lambda exp: 0.9),
        ('Sales Funnel Optimization', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Advanced Revenue Analysis', lambda exp: 0.4 if exp <= 5 else 0.6),
    ],
    'Director of Operations': [
        ('Operations Strategy', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Process Improvement', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Budget Oversight', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Team Coordination', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Leadership', lambda exp: 0.9),
        ('Operations Performance Metrics', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Supply Chain Optimization', lambda exp: 0.4 if exp <= 5 else 0.6),
    ],
    'Principal Data Architect': [
        ('Advanced Data Modeling', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Big Data Architecture', lambda exp: 0.6 if exp <= 5 else 0.9),
        ('Cloud Data Solutions', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Data Governance', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('ETL Optimization', lambda exp: 0.3 if exp <= 5 else 0.7),
        ('Data Lake Architecture', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Data Pipeline Scalability', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Enterprise Data Strategy', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Cloud-Native Data Architectures', lambda exp: 0.3 if exp <= 5 else 0.6),
    ],
    'Sr. DBA': [
        ('Advanced SQL Optimization', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Database Tuning', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Backup and Recovery', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Cloud Database Solutions', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Data Security', lambda exp: 0.3 if exp <= 5 else 0.6),
        ('Distributed Database Management', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Advanced Backup Strategies', lambda exp: 0.4 if exp <= 5 else 0.6),
    ],
    'Sr. Accountant': [
        ('Audit Management', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('Tax Planning', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Budget Strategy', lambda exp: 0.3 if exp <= 5 else 0.7),
        ('Advanced Financial Reporting', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('Leadership', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Forensic Accounting Techniques', lambda exp: 0.3 if exp <= 5 else 0.6),
        ('Advanced Budget Forecasting', lambda exp: 0.4 if exp <= 5 else 0.7),
    ],
    'Software Engineering Manager': [
        ('Agile Development Leadership', lambda exp: 0.7 if exp <= 5 else 0.9),
        ('System Architecture Oversight', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Code Review Practices', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Team Management', lambda exp: 0.4 if exp <= 5 else 0.8),
        ('Process Improvement', lambda exp: 0.3 if exp <= 5 else 0.7),
        ('CI/CD Pipeline Management', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Microservices Architecture Design', lambda exp: 0.4 if exp <= 5 else 0.7),
    ]
}

In [3]:
role_certifications = {
    'Production Technician I': [
        ('Basic Safety Certification', lambda exp: 0.7 if exp <= 5 else 0.5),
        ('OSHA Certification', lambda exp: 0.5 if exp <= 5 else 0.6)
    ],
    'Production Technician II': [
        ('Basic Safety Certification', lambda exp: 0.6 if exp <= 5 else 0.7),
        ('Advanced Machinery Maintenance Certification', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('OSHA Certification', lambda exp: 0.5 if exp <= 5 else 0.7)
    ],
    'Area Sales Manager': [
        ('Salesforce Certified', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Negotiation Specialist Certification', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Production Manager': [
        ('Six Sigma Green Belt', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Lean Manufacturing Certification', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'IT Support': [
        ('CompTIA A+', lambda exp: 0.7 if exp <= 5 else 0.8),
        ('Microsoft Certified: Azure Fundamentals', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Software Engineer': [
        ('AWS Certified Developer - Associate', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('Certified Kubernetes Administrator', lambda exp: 0.3 if exp <= 5 else 0.5),
        ('Google Cloud Professional Developer', lambda exp: 0.2 if exp <= 5 else 0.4)
    ],
    'Data Analyst': [
        ('Tableau Desktop Specialist', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Google Data Analytics Professional Certificate', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Network Engineer': [
        ('Cisco CCNA', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Firewall Specialist Certification', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Database Administrator': [
        ('Oracle Certified Associate', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Microsoft Certified: Azure Database Administrator Associate', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Sales Manager': [
        ('Salesforce Certified Administrator', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Revenue Optimization Specialist Certification', lambda exp: 0.3 if exp <= 5 else 0.5)
    ],
    'Sr. Network Engineer': [
        ('Cisco CCNP', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('AWS Certified Advanced Networking', lambda exp: 0.3 if exp <= 5 else 0.6)
    ],
    'BI Developer': [
        ('Microsoft Power BI Data Analyst', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Tableau Desktop Certified Professional', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Administrative Assistant': [
        ('Administrative Excellence Certification', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Accountant I': [
        ('QuickBooks Certified', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Certified Public Accountant (CPA)', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Enterprise Architect': [
        ('TOGAF Certified', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('AWS Certified Solutions Architect', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'IT Director': [
        ('ITIL Foundation', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Certified Information Systems Security Professional (CISSP)', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'President & CEO': [
        ('Certified Leadership Professional', lambda exp: 0.3 if exp <= 5 else 0.5)
    ],
    'IT Manager - DB': [
        ('Microsoft Certified: Azure Database Administrator Associate', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('AWS Certified Database Specialty', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Senior BI Developer': [
        ('Microsoft Power BI Data Analyst', lambda exp: 0.6 if exp <= 5 else 0.8),
        ('Tableau Desktop Certified Professional', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Google Data Analytics Professional Certificate', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Shared Services Manager': [
        ('Project Management Professional (PMP)', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'CIO': [
        ('Certified Information Systems Security Professional (CISSP)', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('ITIL Expert', lambda exp: 0.5 if exp <= 5 else 0.7)
    ],
    'IT Manager - Support': [
        ('CompTIA Server+', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('Microsoft Certified: Azure Administrator Associate', lambda exp: 0.3 if exp <= 5 else 0.5)
    ],
    'IT Manager - Infra': [
        ('AWS Certified Solutions Architect', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('Certified Kubernetes Administrator', lambda exp: 0.3 if exp <= 5 else 0.5)
    ],
    'Data Architect': [
        ('Google Cloud Professional Data Engineer', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('AWS Certified Big Data Specialty', lambda exp: 0.3 if exp <= 5 else 0.5)
    ],
    'BI Director': [
        ('Microsoft Power BI Data Analyst', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Google Data Analytics Professional Certificate', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Director of Sales': [
        ('Salesforce Certified', lambda exp: 0.5 if exp <= 5 else 0.7)
    ],
    'Director of Operations': [
        ('Six Sigma Black Belt', lambda exp: 0.4 if exp <= 5 else 0.6),
        ('Project Management Professional (PMP)', lambda exp: 0.3 if exp <= 5 else 0.5)
    ],
    'Principal Data Architect': [
        ('Google Cloud Professional Data Engineer', lambda exp: 0.5 if exp <= 5 else 0.8),
        ('AWS Certified Big Data Specialty', lambda exp: 0.4 if exp <= 5 else 0.7)
    ],
    'Sr. DBA': [
        ('Microsoft Certified: Azure Database Administrator Associate', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('AWS Certified Database Specialty', lambda exp: 0.4 if exp <= 5 else 0.6)
    ],
    'Sr. Accountant': [
        ('Certified Public Accountant (CPA)', lambda exp: 0.5 if exp <= 5 else 0.7),
        ('Chartered Financial Analyst (CFA)', lambda exp: 0.3 if exp <= 5 else 0.5)
    ],
    'Software Engineering Manager': [
        ('Certified Kubernetes Administrator', lambda exp: 0.4 if exp <= 5 else 0.7),
        ('AWS Certified Solutions Architect', lambda exp: 0.3 if exp <= 5 else 0.5)
    ]
}

In [4]:
feature_description = {
    # General Features
    "Position_IT Support": "Refers to the previous position held by the candidate in IT support roles, involving troubleshooting and maintaining IT systems (included in the position details of the candidate).",
    "Position_Production Technician I": "Indicates the candidate's prior experience as a Production Technician I, responsible for operating machinery and maintaining production efficiency (included in the position details of the candidate).",
    "Position_Area Sales Manager": "Represents prior experience as an Area Sales Manager, overseeing regional sales operations and driving revenue growth (included in the position details of the candidate).",
    "Position_Production Manager": "Denotes experience as a Production Manager, managing manufacturing processes and ensuring optimal productivity (included in the position details of the candidate).",
    "Position_Production Technician II": "Highlights the candidate's previous role as a Production Technician II, involving advanced machinery operation and troubleshooting (included in the position details of the candidate).",
    "Position_Sales Manager": "Indicates the candidate's prior role as a Sales Manager, leading sales teams and developing sales strategies (included in the position details of the candidate).",
    "Position_Enterprise Architect": "Refers to the candidate's experience as an Enterprise Architect, designing and implementing enterprise-level IT solutions (included in the position details of the candidate).",
    "Position_Network Engineer": "Denotes the candidate's experience as a Network Engineer, managing and optimizing network infrastructure (included in the position details of the candidate).",
    "Position_Sr. Network Engineer": "Represents prior experience as a Senior Network Engineer, handling complex network configurations and troubleshooting (included in the position details of the candidate).",
    "Position_Database Administrator": "Highlights experience as a Database Administrator, managing databases and ensuring data integrity and performance (included in the position details of the candidate).",
    "Position_Data Analyst": "Indicates the candidate's role as a Data Analyst, interpreting and visualizing data for business insights (included in the position details of the candidate).",
    "Position_Software Engineer": "Refers to the candidate's experience as a Software Engineer, developing and maintaining software applications (included in the position details of the candidate).",
    "Position_Sr. DBA": "Denotes prior experience as a Senior Database Administrator, overseeing database operations and strategy (included in the position details of the candidate).",
    "Position_Sr. Accountant": "Represents the candidate's experience as a Senior Accountant, handling advanced accounting and financial reporting tasks (included in the position details of the candidate).",
    "Position_Administrative Assistant": "Highlights the candidate's role as an Administrative Assistant, supporting office operations and administrative tasks (included in the position details of the candidate).",
    "Position_Accountant I": "Indicates the candidate's role as an entry-level accountant, performing basic accounting tasks (included in the position details of the candidate).",
    "Position_Shared Services Manager": "Refers to experience managing shared services operations, focusing on efficiency and resource optimization (included in the position details of the candidate).",
    "Position_IT Director": "Denotes prior experience as an IT Director, leading IT strategy and operations for an organization (included in the position details of the candidate).",
    "Position_CIO": "Represents experience as a Chief Information Officer, overseeing all IT-related initiatives and strategy (included in the position details of the candidate).",
    "Position_Principal Data Architect": "Highlights experience as a Principal Data Architect, designing large-scale data architectures (included in the position details of the candidate).",
    "Position_IT Manager - DB": "Indicates the candidate's role as an IT Manager specializing in database management and optimization (included in the position details of the candidate).",
    "Position_IT Manager - Support": "Refers to the candidate's role as an IT Manager focused on IT support operations and team leadership (included in the position details of the candidate).",
    "Position_IT Manager - Infra": "Denotes the candidate's role as an IT Manager overseeing infrastructure development and maintenance (included in the position details of the candidate).",
    "Position_BI Developer": "Represents experience as a Business Intelligence Developer, creating dashboards and data-driven solutions (included in the position details of the candidate).",
    "Position_Senior BI Developer": "Highlights the candidate's advanced role as a Senior BI Developer, designing sophisticated BI systems (included in the position details of the candidate).",
    "Position_Data Architect": "Indicates the candidate's role as a Data Architect, developing data models and structures (included in the position details of the candidate).",
    "Position_BI Director": "Refers to experience as a Business Intelligence Director, leading BI strategy and team efforts (included in the position details of the candidate).",
    "Position_Director of Sales": "Denotes prior experience as a Director of Sales, managing and leading sales operations (included in the position details of the candidate).",
    "Position_Director of Operations": "Represents experience as a Director of Operations, overseeing day-to-day operations and strategy execution (included in the position details of the candidate).",
    "Position_Software Engineering Manager": "Highlights the candidate's role as a Software Engineering Manager, leading development teams and projects (included in the position details of the candidate).",
    "Position_President & CEO": "Indicates experience as a President or CEO, responsible for organizational leadership and strategy (included in the position details of the candidate).",
    "State": "Refers to the state in which the candidate is located, providing geographical context for the profile.",
    "Sex": "Indicates the gender of the candidate.",
    "CitizenDesc_US Citizen": "Indicates that the candidate is a United States citizen (included in the citizenship details of the candidate).",
    "CitizenDesc_Eligible NonCitizen": "Represents the candidate's status as an eligible non-citizen (included in the citizenship details of the candidate).",
    "CitizenDesc_Non-Citizen": "Indicates that the candidate is not a U.S. citizen (included in the citizenship details of the candidate).",
    "HispanicLatino": "Represents whether the candidate identifies as Hispanic or Latino.",
    "RaceDesc_White": "Indicates the candidate identifies as White.",
    "RaceDesc_Black or African American": "Indicates the candidate identifies as Black or African American.",
    "RaceDesc_Asian": "Indicates the candidate identifies as Asian.",
    "RaceDesc_American Indian or Alaska Native": "Indicates the candidate identifies as American Indian or Alaska Native.",
    "RaceDesc_Hispanic": "Indicates the candidate identifies as Hispanic.",
    "RaceDesc_Two or more races": "Indicates the candidate identifies as having two or more racial identities.",
    "Department_IT/IS": "Indicates that the candidate has prior experience in the IT/IS department (included in the department details of the candidate).",
    "Department_Production": "Represents the candidate's previous experience in the Production department (included in the department details of the candidate).",
    "Department_Sales": "Highlights the candidate's previous experience in the Sales department (included in the department details of the candidate).",
    "Department_Software Engineering": "Indicates the candidate's experience in the Software Engineering department (included in the department details of the candidate).",
    "Department_Admin Offices": "Refers to the candidate's experience in administrative office roles (included in the department details of the candidate).",
    "Department_Executive Office": "Represents the candidate's experience in executive-level roles (included in the department details of the candidate).",
    "YearsExperience": "Indicates the total number of years of experience the candidate has (included in the candidate's professional background).",
    "AgeGroup": "Represents the age group of the candidate, providing additional demographic context.",
    "ExperienceCategory": "Classifies the candidate's experience into predefined categories based on their years of experience.",
    "Education": "Indicates the level of education attained by the candidate, such as Bachelor's or Master's degree (included in the candidate's academic background).",
    # Technical skills
    "Basic Machinery Maintenance": "Ensures the upkeep of machinery for smooth operations, critical for production efficiency (included in the technical skills rating of the candidate).",
    "Safety Protocols": "Maintains a safe working environment by adhering to industry-standard safety practices, vital for worker protection (included in the technical skills rating of the candidate).",
    "Problem Identification": "Recognizes issues promptly to minimize downtime and optimize productivity (included in the technical skills rating of the candidate).",
    "Advanced Machinery Troubleshooting": "Resolves complex machinery issues efficiently to maintain operational continuity (included in the technical skills rating of the candidate).",
    "Teamwork": "Collaborates effectively with others to achieve team objectives (included in the technical skills rating of the candidate).",
    "Advanced Machinery Maintenance": "Ensures long-term reliability of machinery through expert maintenance (included in the technical skills rating of the candidate).",
    "Problem-Solving": "Develops and implements solutions to production challenges, emphasizing critical thinking (included in the technical skills rating of the candidate).",
    "Efficiency Optimization": "Improves processes to increase productivity and reduce costs (included in the technical skills rating of the candidate).",
    "Leadership Skills": "Guides teams effectively to achieve goals and maintain motivation (included in the technical skills rating of the candidate).",
    "Preventive Maintenance Planning": "Plans maintenance schedules to avoid breakdowns and ensure consistent operations (included in the technical skills rating of the candidate).",
    "Sales Strategy": "Designs and implements strategies to achieve sales targets and business growth (included in the technical skills rating of the candidate).",
    "Negotiation": "Engages in discussions to reach mutually beneficial agreements, critical in sales and management roles (included in the technical skills rating of the candidate).",
    "Customer Relationship Management": "Manages and nurtures customer relationships to enhance satisfaction and retention (included in the technical skills rating of the candidate).",
    "Market Analysis": "Analyzes market trends to identify opportunities and inform decision-making (included in the technical skills rating of the candidate).",
    "Team Leadership": "Leads teams to achieve objectives and maintain high performance (included in the technical skills rating of the candidate).",
    "Python": "A versatile programming language used for developing scalable solutions and models (included in the technical skills rating of the candidate).",
    "SQL": "A crucial language for managing and querying databases in data-driven roles (included in the technical skills rating of the candidate).",
    "Data Visualization": "Transforms complex data into clear visual insights for decision-making (included in the technical skills rating of the candidate).",
    "Statistical Analysis": "Applies statistical methods to extract actionable insights from data (included in the technical skills rating of the candidate).",
    "Network Configuration": "Designs and manages network infrastructure for efficient and secure connectivity (included in the technical skills rating of the candidate).",
    "Firewall Management": "Configures and monitors firewalls to protect systems from unauthorized access (included in the technical skills rating of the candidate).",
    "System Architecture Design": "Designs efficient and scalable system structures to meet organizational needs (included in the technical skills rating of the candidate).",
    "Strategic Vision": "Develops a long-term vision aligned with business objectives to drive innovation (included in the technical skills rating of the candidate).",
    "IT Governance": "Establishes policies and procedures to ensure IT aligns with organizational goals (included in the technical skills rating of the candidate).",
    "Troubleshooting": "Diagnoses and resolves technical issues to ensure smooth operations (included in the technical skills rating of the candidate).",
    "ETL Development": "Designs and optimizes data extraction, transformation, and loading processes (included in the technical skills rating of the candidate).",
    "Document Management": "Organizes and maintains critical documents for operational efficiency (included in the technical skills rating of the candidate).",
    "Database Tuning": "Optimizes database performance to ensure efficient querying and data management (included in the technical skills rating of the candidate).",
    "Database Management": "Oversees and maintains databases to ensure data integrity, availability, and reliability (included in the technical skills rating of the candidate).",
    "Data Governance": "Defines policies and procedures to ensure proper data management and compliance (included in the technical skills rating of the candidate).",
    "Governance and Standards": "Establishes frameworks to ensure systems align with organizational and regulatory standards (included in the technical skills rating of the candidate).",
    "Big Data Solutions": "Designs and implements scalable solutions for processing and analyzing large datasets (included in the technical skills rating of the candidate).",
    "Data Lake Architecture": "Creates frameworks for storing and managing vast amounts of raw and processed data (included in the technical skills rating of the candidate).",
    "Budget Strategy": "Plans and manages financial resources to ensure efficient allocation and cost control (included in the technical skills rating of the candidate).",
    "Advanced ITSM Tools": "Utilizes advanced IT service management tools to enhance operational efficiency (included in the technical skills rating of the candidate).",
    "Data Security": "Implements measures to protect sensitive data from breaches and unauthorized access (included in the technical skills rating of the candidate).",
    "Advanced CRM Tools": "Uses sophisticated customer relationship management tools to enhance client interactions (included in the technical skills rating of the candidate).",
    "Backup Strategies": "Develops plans to secure data through regular backups and recovery methods (included in the technical skills rating of the candidate).",
    "Lean Manufacturing": "Applies principles to minimize waste and maximize efficiency in production processes (included in the technical skills rating of the candidate).",
    "Microservices Architecture Design": "Designs modular, scalable software architectures to enhance flexibility and efficiency (included in the technical skills rating of the candidate).",
    "Advanced Financial Reporting": "Prepares detailed and insightful financial reports for informed decision-making (included in the technical skills rating of the candidate).",
    "SD-WAN Deployment": "Implements software-defined wide-area network solutions to optimize connectivity (included in the technical skills rating of the candidate).",
    "Revenue Optimization": "Develops strategies to maximize income while minimizing costs (included in the technical skills rating of the candidate).",
    "Budget Oversight": "Manages and monitors budgets to ensure financial targets are met (included in the technical skills rating of the candidate).",
    "Business-IT Alignment": "Ensures that IT strategies and operations are aligned with business goals (included in the technical skills rating of the candidate).",
    "Network Management": "Maintains and optimizes networks for seamless connectivity and performance (included in the technical skills rating of the candidate).",
    "Advanced Troubleshooting Techniques": "Applies expert-level problem-solving methods to resolve complex issues (included in the technical skills rating of the candidate).",
    "Process Optimization": "Analyzes and refines processes to improve efficiency and effectiveness (included in the technical skills rating of the candidate).",
    "Hardware Maintenance": "Performs regular maintenance to ensure hardware operates at peak performance (included in the technical skills rating of the candidate).",
    "Advanced Data Visualization": "Creates detailed and intuitive visual representations of complex data (included in the technical skills rating of the candidate).",
    "Troubleshooting Oversight": "Supervises and resolves operational and technical issues effectively (included in the technical skills rating of the candidate).",
    "VPN Setup": "Configures virtual private networks to ensure secure remote access (included in the technical skills rating of the candidate).",
    "Disaster Recovery Planning": "Develops strategies to recover operations quickly after disruptions (included in the technical skills rating of the candidate).",
    "Big Data Architecture": "Designs infrastructure for processing and storing large-scale data systems (included in the technical skills rating of the candidate).",
    "Data Pipeline Optimization": "Enhances data pipelines for improved speed and reliability (included in the technical skills rating of the candidate).",
    "Cloud Database Solutions": "Implements cloud-based database systems for scalable data management (included in the technical skills rating of the candidate).",
    "Customer Support Strategies": "Develops approaches to ensure exceptional customer service and satisfaction (included in the technical skills rating of the candidate).",
    "Real-Time Data Processing": "Processes and analyzes data in real-time for immediate insights (included in the technical skills rating of the candidate).",
    "Cloud Networking": "Configures and manages networks hosted on cloud platforms (included in the technical skills rating of the candidate).",
    "Public Relations": "Manages communication strategies to build and maintain a positive public image (included in the technical skills rating of the candidate).",
    "SQL Optimization": "Improves database query efficiency to enhance performance (included in the technical skills rating of the candidate).",
    "Customer Retention": "Implements strategies to maintain and grow long-term customer relationships (included in the technical skills rating of the candidate).",
    "Vendor Management": "Oversees supplier relationships to ensure quality and cost efficiency (included in the technical skills rating of the candidate).",
    "Advanced Backup Strategies": "Designs and implements advanced data backup processes for security and reliability (included in the technical skills rating of the candidate).",
    "Team Management": "Leads and organizes teams to achieve objectives and ensure productivity (included in the technical skills rating of the candidate).",
    "Data Pipeline Scalability": "Builds scalable data pipelines to handle increasing data loads efficiently (included in the technical skills rating of the candidate).",
    "IT Security Oversight": "Supervises security measures to protect organizational IT systems (included in the technical skills rating of the candidate).",
    "Advanced Visualization": "Uses advanced tools to create impactful and detailed visualizations (included in the technical skills rating of the candidate).",
    "Business Intelligence Tools": "Leverages tools to gather, analyze, and visualize business data (included in the technical skills rating of the candidate).",
    "Budget Planning": "Plans and allocates budgets to align with organizational goals (included in the technical skills rating of the candidate).",
    "Software Design": "Designs software systems to meet user requirements and business goals (included in the technical skills rating of the candidate).",
    "Advanced Network Configuration": "Configures complex network systems to meet advanced organizational needs (included in the technical skills rating of the candidate).",
    "Network Security Design": "Designs secure network systems to protect against cyber threats (included in the technical skills rating of the candidate).",
    "Predictive Analytics Integration": "Incorporates predictive analytics into business processes for proactive decision-making (included in the technical skills rating of the candidate).",
    "Team Coordination": "Aligns team efforts to achieve project goals efficiently (included in the technical skills rating of the candidate).",
    "Machine Learning Integration": "Integrates machine learning models into operational systems for intelligent solutions (included in the technical skills rating of the candidate).",
    "Cloud-Native Data Architectures": "Designs data architectures optimized for cloud platforms (included in the technical skills rating of the candidate).",
    "Financial Reporting": "Prepares accurate financial statements to support organizational decision-making (included in the technical skills rating of the candidate).",
    "Enterprise Data Strategy": "Defines strategic approaches to managing and leveraging enterprise data (included in the technical skills rating of the candidate).",
    "Advanced SQL Optimization": "Utilizes advanced techniques to enhance database query performance (included in the technical skills rating of the candidate).",
    "Tax Preparation": "Prepares accurate and compliant tax filings for individuals or organizations (included in the technical skills rating of the candidate).",
    "ETL Optimization": "Refines ETL processes to improve data extraction, transformation, and loading efficiency (included in the technical skills rating of the candidate).",
    "Sales Funnel Optimization": "Improves sales funnel processes to maximize conversion rates (included in the technical skills rating of the candidate).",
    "Network Performance Optimization": "Optimizes network settings to enhance overall performance and reliability (included in the technical skills rating of the candidate).",
    "Firewall Expertise": "Manages and configures firewalls to ensure robust system security (included in the technical skills rating of the candidate).",
    "Cybersecurity Oversight": "Supervises strategies and implementations to safeguard digital assets (included in the technical skills rating of the candidate).",
    "Financial Management": "Manages finances to ensure alignment with strategic objectives (included in the technical skills rating of the candidate).",
    "Technology Roadmap Development": "Develops long-term plans to guide technological advancements (included in the technical skills rating of the candidate).",
    "Operations Performance Metrics": "Defines and tracks key metrics to assess operational success (included in the technical skills rating of the candidate).",
    "Forensic Accounting Techniques": "Applies investigative methods to analyze financial discrepancies (included in the technical skills rating of the candidate).",
    "Customer Support": "Addresses customer needs and resolves issues to ensure satisfaction (included in the technical skills rating of the candidate).",
    "Code Review Practices": "Establishes and performs systematic reviews of code for quality and consistency (included in the technical skills rating of the candidate).",
    "Cloud Data Solutions": "Implements cloud-based systems to manage and process data efficiently (included in the technical skills rating of the candidate).",
    "QuickBooks": "Uses QuickBooks for managing financial data and transactions (included in the technical skills rating of the candidate).",
    "Advanced Firewall Configurations": "Sets up and maintains firewalls to handle complex security needs (included in the technical skills rating of the candidate).",
    "Infrastructure Design": "Creates and plans infrastructure to support organizational IT needs (included in the technical skills rating of the candidate).",
    "Cloud Data Management": "Manages cloud data storage and access to ensure availability and security (included in the technical skills rating of the candidate).",
    "Quality Assurance": "Ensures processes and outputs meet established quality standards (included in the technical skills rating of the candidate).",
    "Basic Accounting": "Handles fundamental accounting tasks to manage financial records (included in the technical skills rating of the candidate).",
    "Advanced Budget Forecasting": "Uses advanced techniques to predict future financial performance (included in the technical skills rating of the candidate).",
    "Strategic Planning": "Develops long-term plans to achieve business objectives (included in the technical skills rating of the candidate).",
    "Risk Assessment": "Identifies and evaluates risks to prepare mitigation strategies (included in the technical skills rating of the candidate).",
    "Performance Tuning": "Optimizes system performance for maximum efficiency (included in the technical skills rating of the candidate).",
    "Machine Learning": "Develops and applies machine learning models for intelligent automation (included in the technical skills rating of the candidate).",
    "Incident Response Planning": "Creates strategies to respond effectively to IT incidents (included in the technical skills rating of the candidate).",
    "Leadership": "Guides teams and initiatives to achieve strategic goals (included in the technical skills rating of the candidate).",
    "Cost Reduction Techniques": "Implements methods to minimize costs without compromising quality (included in the technical skills rating of the candidate).",
    "Agile Development Leadership": "Leads teams using Agile practices to enhance software development (included in the technical skills rating of the candidate).",
    "Operations Strategy": "Defines strategies to optimize operational processes and efficiency (included in the technical skills rating of the candidate).",
    "Advanced Predictive Modeling": "Uses advanced techniques to develop accurate predictive models (included in the technical skills rating of the candidate).",
    "Process Improvement": "Analyzes and improves processes for better efficiency and outcomes (included in the technical skills rating of the candidate).",
    "Service Delivery Optimization": "Enhances service delivery processes for customer satisfaction (included in the technical skills rating of the candidate).",
    "Strategic IT Investment Planning": "Plans IT investments to align with strategic business goals (included in the technical skills rating of the candidate).",
    "Competitor Analysis": "Examines competitors to identify market opportunities and threats (included in the technical skills rating of the candidate).",
    "Office Coordination": "Organizes office operations to ensure efficiency and productivity (included in the technical skills rating of the candidate).",
    "Hardware Management": "Oversees hardware to ensure proper functioning and availability (included in the technical skills rating of the candidate).",
    "Cloud Integration": "Integrates cloud services into organizational IT frameworks (included in the technical skills rating of the candidate).",
    "Customer Communication": "Communicates effectively with customers to build strong relationships (included in the technical skills rating of the candidate).",
    "System Troubleshooting": "Identifies and resolves system issues to maintain smooth operations (included in the technical skills rating of the candidate).",
    "Supply Chain Optimization": "Streamlines supply chain processes to enhance efficiency and reduce costs (included in the technical skills rating of the candidate).",
    "ETL Automation": "Automates data extraction, transformation, and loading processes for efficiency (included in the technical skills rating of the candidate).",
    "System Architecture Oversight": "Supervises the design and development of system architectures (included in the technical skills rating of the candidate).",
    "Dashboard Creation": "Designs dashboards to present data insights effectively (included in the technical skills rating of the candidate).",
    "Java": "A versatile programming language used for building robust applications (included in the technical skills rating of the candidate).",
    "Advanced Data Modeling": "Designs complex data models to represent organizational data structures (included in the technical skills rating of the candidate).",
    "Data Modeling": "Creates data models to structure and organize information effectively (included in the technical skills rating of the candidate).",
    "Business Intelligence Strategy": "Develops strategic approaches to leverage BI tools and insights (included in the technical skills rating of the candidate).",
    "System Architecture": "Designs system architectures to meet technical and business requirements (included in the technical skills rating of the candidate).",
    "Backup and Recovery": "Implements plans to secure data and restore operations after failures (included in the technical skills rating of the candidate).",
    "Distributed Database Management": "Manages databases distributed across multiple locations for scalability (included in the technical skills rating of the candidate).",
    "Production Line Efficiency Analysis": "Analyzes production processes to improve line efficiency (included in the technical skills rating of the candidate).",
    "Hybrid Cloud Infrastructure Management": "Oversees hybrid cloud systems to combine local and cloud resources efficiently (included in the technical skills rating of the candidate).",
    "System Upgrades": "Plans and executes system upgrades to improve functionality and performance (included in the technical skills rating of the candidate).",
    "Audit Assistance": "Supports audits by providing necessary documentation and insights (included in the technical skills rating of the candidate).",
    "Solution Architecture": "Designs solutions to meet technical and business challenges effectively (included in the technical skills rating of the candidate).",
    "IT Support Management": "Oversees IT support processes to ensure effective resolution of technical issues (included in the technical skills rating of the candidate).",
    "CI/CD Pipeline Management": "Manages CI/CD pipelines to streamline software development and deployment (included in the technical skills rating of the candidate).",
    "Tax Planning": "Develops strategies for efficient and compliant tax planning (included in the technical skills rating of the candidate).",
    "Audit Management": "Manages audits to ensure compliance and identify operational improvements (included in the technical skills rating of the candidate).",
    "Cloud Strategy": "Develops strategies to leverage cloud technologies for business growth (included in the technical skills rating of the candidate).",
    "Database Design": "Creates and structures databases for optimized data storage and retrieval (included in the technical skills rating of the candidate).",
    "Advanced Revenue Analysis": "Performs detailed analysis to identify and maximize revenue streams (included in the technical skills rating of the candidate).",
    "Scheduling": "Plans and manages schedules to ensure timely completion of tasks (included in the technical skills rating of the candidate).",
    # Certifications
    "Basic Safety Certification": "Covers essential safety practices to ensure a secure working environment (included in the certification rating of the candidate).",
    "OSHA Certification": "Ensures compliance with workplace safety and health regulations (included in the certification rating of the candidate).",
    "Advanced Machinery Maintenance Certification": "Validates advanced skills in maintaining and repairing machinery for improved efficiency (included in the certification rating of the candidate).",
    "Salesforce Certified": "Demonstrates proficiency in using Salesforce for customer relationship management (included in the certification rating of the candidate).",
    "Negotiation Specialist Certification": "Certifies advanced negotiation skills for successful deal-making (included in the certification rating of the candidate).",
    "Six Sigma Green Belt": "Recognizes expertise in process improvement and waste reduction using Six Sigma methodologies (included in the certification rating of the candidate).",
    "Lean Manufacturing Certification": "Validates knowledge of lean manufacturing principles to enhance production efficiency (included in the certification rating of the candidate).",
    "CompTIA A+": "Certifies foundational IT knowledge and troubleshooting skills (included in the certification rating of the candidate).",
    "Microsoft Certified: Azure Fundamentals": "Validates basic knowledge of Azure cloud services and solutions (included in the certification rating of the candidate).",
    "AWS Certified Developer - Associate": "Certifies skills in developing and maintaining AWS-based applications (included in the certification rating of the candidate).",
    "Certified Kubernetes Administrator": "Recognizes expertise in managing Kubernetes clusters and deployments (included in the certification rating of the candidate).",
    "Google Cloud Professional Developer": "Certifies advanced skills in developing and managing applications on Google Cloud (included in the certification rating of the candidate).",
    "Tableau Desktop Specialist": "Validates foundational skills in creating Tableau dashboards and data visualizations (included in the certification rating of the candidate).",
    "Google Data Analytics Professional Certificate": "Covers essential skills for data analysis, including visualization and SQL (included in the certification rating of the candidate).",
    "Cisco CCNA": "Certifies networking knowledge and skills for managing and troubleshooting networks (included in the certification rating of the candidate).",
    "Firewall Specialist Certification": "Demonstrates expertise in configuring and managing firewalls for network security (included in the certification rating of the candidate).",
    "Oracle Certified Associate": "Certifies foundational database management and SQL skills (included in the certification rating of the candidate).",
    "Microsoft Certified: Azure Database Administrator Associate": "Validates database administration skills on Azure cloud platforms (included in the certification rating of the candidate).",
    "Salesforce Certified Administrator": "Demonstrates expertise in managing Salesforce platforms and automation (included in the certification rating of the candidate).",
    "Revenue Optimization Specialist Certification": "Certifies skills in improving revenue streams and maximizing profitability (included in the certification rating of the candidate).",
    "Cisco CCNP": "Recognizes advanced networking skills for configuring and managing enterprise networks (included in the certification rating of the candidate).",
    "AWS Certified Advanced Networking": "Validates expertise in designing and managing advanced networking solutions on AWS (included in the certification rating of the candidate).",
    "Microsoft Power BI Data Analyst": "Certifies skills in creating data visualizations and analytical reports using Power BI (included in the certification rating of the candidate).",
    "Tableau Desktop Certified Professional": "Recognizes advanced proficiency in building Tableau dashboards and analyses (included in the certification rating of the candidate).",
    "Administrative Excellence Certification": "Validates skills in organizing and managing administrative tasks efficiently (included in the certification rating of the candidate).",
    "QuickBooks Certified": "Certifies proficiency in using QuickBooks for financial management and accounting (included in the certification rating of the candidate).",
    "Certified Public Accountant (CPA)": "Recognizes advanced expertise in financial accounting and reporting (included in the certification rating of the candidate).",
    "TOGAF Certified": "Certifies skills in enterprise architecture framework and design (included in the certification rating of the candidate).",
    "AWS Certified Solutions Architect": "Validates expertise in designing scalable and secure cloud architectures (included in the certification rating of the candidate).",
    "ITIL Foundation": "Certifies foundational knowledge in IT service management practices (included in the certification rating of the candidate).",
    "Certified Information Systems Security Professional (CISSP)": "Recognizes expertise in managing and securing IT systems (included in the certification rating of the candidate).",
    "Certified Leadership Professional": "Validates leadership skills for managing teams and driving business outcomes (included in the certification rating of the candidate).",
    "AWS Certified Database Specialty": "Certifies advanced knowledge in designing and managing cloud databases on AWS (included in the certification rating of the candidate).",
    "Project Management Professional (PMP)": "Certifies expertise in managing projects effectively and meeting strategic objectives (included in the certification rating of the candidate).",
    "ITIL Expert": "Recognizes advanced knowledge of IT service management frameworks and strategies (included in the certification rating of the candidate).",
    "CompTIA Server+": "Certifies skills in managing and troubleshooting servers (included in the certification rating of the candidate).",
    "Google Cloud Professional Data Engineer": "Certifies expertise in designing and managing scalable data systems on Google Cloud (included in the certification rating of the candidate).",
    "AWS Certified Big Data Specialty": "Validates advanced skills in handling and analyzing big data on AWS platforms (included in the certification rating of the candidate).",
    "Six Sigma Black Belt": "Recognizes advanced expertise in process improvement and quality management using Six Sigma methodologies (included in the certification rating of the candidate).",
    "Chartered Financial Analyst (CFA)": "Certifies expertise in financial analysis and investment management (included in the certification rating of the candidate).",
    "Microsoft Certified: Azure Administrator Associate": "Validates skills in managing Azure cloud solutions and resources (included in the certification rating of the candidate).",
    "Certified Kubernetes Administrator": "Recognizes expertise in managing Kubernetes clusters and deployments (included in the certification rating of the candidate)."
}

In [5]:
# Count occurrences of each key in feature_description
feature_description_keys = list(feature_description.keys())
duplicates = [key for key, count in Counter(feature_description_keys).items() if count > 1]

print("Keys that are duplicated in feature_description:")
print(duplicates)

Keys that are duplicated in feature_description:
[]


In [6]:
# Flatten the skill names from role_skills
role_skills_keys = set(skill for role in role_skills.values() for skill, _ in role)

# Get the keys from feature_description
feature_description_keys = set(feature_description.keys())

# Find the keys in role_skills but not in feature_description
missing_keys = role_skills_keys - feature_description_keys

print("Keys in role_skills but not in feature_description:")
print(missing_keys)

Keys in role_skills but not in feature_description:
set()


In [7]:
# Flatten the skill names from role_certifications
role_certifications_keys = set(skill for role in role_certifications.values() for skill, _ in role)

# Get the keys from feature_description
feature_description_keys = set(feature_description.keys())

# Find the keys in role_skills but not in feature_description
missing_keys = role_certifications_keys - feature_description_keys

print("Keys in role_certifications but not in feature_description:")
print(missing_keys)

Keys in role_certifications but not in feature_description:
set()


In [8]:
# Define the directory and file path
directory = "../app/data/"
file_path = os.path.join(directory, "feature_description.json")

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the feature_description dictionary as a JSON file
with open(file_path, "w") as file:
    json.dump(feature_description, file, indent=4)

print(f"File saved at: {file_path}")

File saved at: ../app/data/feature_description.json
